In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary  # Denne er fin for å printe ut informasjon om PyTorch-modeller
import matplotlib.pyplot as plt
from collections import Counter
from code_base.functions import train, train_models, select_best_model, evaluate_performance
from code_base.models import TestNet, LeNetVariant, ObjectDetect_2x3
from code_base.DataAnalysis import DataAnalysis
from code_base.object_detection import plot_instances, get_converted_data

In [ ]:
SEED = 265
torch.manual_seed(SEED)

DEVICE = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Using device: {DEVICE}")

# Object Localization

## Data exploration

In [ ]:
# Loading the data as PyTorch Tensor Dataset
data_train = torch.load("data/localization_train.pt")
data_val = torch.load("data/localization_val.pt")
data_test = torch.load("data/localization_test.pt")
print("Type: ", type(data_train))

In [ ]:
DataAnalysis.get_summary(data_train)

In [ ]:
DataAnalysis.get_summary(data_val)

In [ ]:
DataAnalysis.get_summary(data_test)

In [ ]:
# Display 4 instances of each class
n_classes = len(Counter([int(label[-1]) for _, label in data_train]))
for i in range(n_classes):
    DataAnalysis.plot_instances_with_bounding_box(data_train, i, n_instances=4)
# No class
DataAnalysis.plot_instances_with_bounding_box(data_train, None, n_instances=4)

## Training

In [ ]:
# Normalize from training data
imgs = torch.stack([img for img, _ in data_train])

# Define normalizer
normalizer = transforms.Normalize(
    imgs.mean(dim=(0, 2, 3)), 
    imgs.std(dim=(0, 2, 3))
    )

data_train_norm = [(normalizer(img), label) for img, label in data_train]
data_val_norm = [(normalizer(img), label) for img, label in data_val]
data_test_norm = [(normalizer(img), label) for img, label in data_test]

In [ ]:
batch_size = 32
train_loader = DataLoader(data_train_norm, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(data_val_norm, batch_size=batch_size, shuffle=False)

In [ ]:
epochs = 15
networks = [TestNet]

hyper_parameters = [
    {"lr": 0.001, "weight_decay": 0.0, "momentum": 0.0},
    # {"lr": 0.01, "weight_decay": 0.01, "momentum": 0.0},
    # {"lr": 0.01, "weight_decay": 0.9, "momentum": 0.0},
]

In [ ]:
trained_models, train_performances, val_performances, train_losses, val_losses = train_models(
    "localization",
    networks,
    hyper_parameters,
    batch_size,
    epochs,
    train_loader,
    val_loader,
    DEVICE,
    SEED)

In [ ]:
best_model, best_model_idx = select_best_model(trained_models, val_performances)
print(best_model)

In [ ]:
best_model_train_loss = train_losses[best_model_idx]
best_model_val_loss = val_losses[best_model_idx]
best_model_train_performance = train_performances[best_model_idx]
best_model_val_performance = val_performances[best_model_idx]
DataAnalysis.plot_performance_over_time(best_model_train_loss, best_model_val_loss, "Training vs val loss")

In [ ]:
DataAnalysis.plot_performance_over_time(best_model_train_performance, best_model_val_performance, "Training accuracy and validation accuracy over epochs", 
                               label1="Training Accuracy", label2="Validation Accuracy")

## Evaluation

In [ ]:
test_loader = DataLoader(data_test_norm, batch_size=batch_size, shuffle=False)

perf, output = evaluate_performance("localization", best_model, test_loader, device=DEVICE)
print(f"Test performance: {perf*100:.2f}%")

In [ ]:
for i in range(10):
    DataAnalysis.plot_instances_with_bounding_box(data_test, i, predictions=output)
DataAnalysis.plot_instances_with_bounding_box(data_test, None, predictions=output)

# Object Detection

#### Imports and setup (again) 
Identical to the imports at the top, they are there simply to make running the object detection part easier.

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary  # Denne er fin for å printe ut informasjon om PyTorch-modeller
import matplotlib.pyplot as plt
from collections import Counter
from code_base.functions import train, train_models, select_best_model, evaluate_performance
from code_base.models import TestNet, LeNetVariant, ObjectDetect_2x3
from code_base.DataAnalysis import DataAnalysis
from code_base.object_detection import plot_instances, get_converted_data

AttributeError: module 'torch' has no attribute 'version'

In [ ]:
SEED = 265
torch.manual_seed(SEED)

DEVICE = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Using device: {DEVICE}")

## Data exploration

In [ ]:
grid_dimensions = (2,3)

In [ ]:
object_train, object_val, object_test = get_converted_data(grid_dimensions=grid_dimensions) 

In [ ]:
plot_instances(object_train, 4, grid_dimensions=grid_dimensions)

## Training

In [ ]:
# Normalize from training data
imgs = torch.stack([img for img, _ in object_train])

# Define normalizer
normalizer = transforms.Normalize(
    imgs.mean(dim=(0, 2, 3)), 
    imgs.std(dim=(0, 2, 3))
    )

object_train_norm = [(normalizer(img), label) for img, label in object_train]
object_val_norm = [(normalizer(img), label) for img, label in object_val]
object_test_norm = [(normalizer(img), label) for img, label in object_test]

In [ ]:
batch_size = 32
train_loader = DataLoader(object_train_norm, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(object_val_norm, batch_size=batch_size, shuffle=False)

In [ ]:
epochs = 5
networks = [ObjectDetect_2x3]

hyper_parameters = [
    {"lr": 0.001, "weight_decay": 0.0, "momentum": 0.0},
    # {"lr": 0.01, "weight_decay": 0.01, "momentum": 0.0},
    # {"lr": 0.01, "weight_decay": 0.9, "momentum": 0.0},
]

In [ ]:
trained_models, train_performances, val_performances, train_losses, val_losses = train_models(
    "detection",
    networks,
    hyper_parameters,
    batch_size,
    epochs,
    train_loader,
    val_loader,
    DEVICE,
    SEED)